# Historical NOAA weather data

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import datetime as dt

In [3]:
import altair as alt
import altair_latimes as lat
import numpy as np

In [4]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")

ThemeRegistry.enable('latimes')

In [5]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.max_colwidth = None

### Get LAX data from the Integrated Surface Dataset

In [48]:
### LAX station
usaf = "722950"
wban = "23174"

In [60]:
# urls = []

# for y in range(1980, 2022):
#     urls.append(
#         "https://www.ncei.noaa.gov/pub/data/noaa/isd-lite/"
#         + f"{y}/"
#         + f"{usaf}-"
#         + f"{wban}-"
#         + f"{y}.gz"
#     )
#     for u in urls:
#         !wget -O 'data/raw/isd/{y}_data.gz' '{u}'

In [61]:
# !gzip -d data/raw/isd/*_data.gz

In [66]:
### Format file: https://www1.ncdc.noaa.gov/pub/data/noaa/isd-lite/isd-lite-format.txt

In [140]:
df = pd.read_fwf(
    "data/raw/isd/1980_data",
    names=[
        "year",
        "month",
        "day",
        "hour",
        "air_temp",
        "dew_temp",
        "sea_level_pressure",
        "wind_direction",
        "wind_speed_rate",
        "sky_condition",
        "precip_1hour",
        "precip_6hour",
    ],
)

In [141]:
df = df.replace(-9999, 0)

In [142]:
df["air_temp"] = (((df["air_temp"] * 1.8) + 32) / 10).round()
df["dew_temp"] = (((df["dew_temp"] * 1.8) + 32) / 10).round()

In [143]:
df["sea_level_pressure"] = df["sea_level_pressure"].astype(float) / 10
df["wind_direction"] = df["wind_direction"].astype(float) / 10
df["wind_speed_rate"] = df["wind_speed_rate"].astype(float) / 10
df["sky_condition"] = df["sky_condition"].astype(float) / 10
df["precip_1hour"] = df["precip_1hour"].astype(float) / 10
df["precip_6hour"] = df["precip_6hour"].astype(float) / 10

In [144]:
import datetime

In [145]:
df["datetime"] = df[["year", "month", "day", "hour"]].apply(
    lambda s: datetime.datetime(*s), axis=1
)

In [146]:
val_cols = [
    "air_temp",
    "dew_temp",
    "sea_level_pressure",
    "wind_direction",
    "wind_speed_rate",
    "sky_condition",
    "precip_1hour",
    "precip_6hour",
]

In [150]:
df[df.precip_6hour == df.precip_6hour.max()]

,year,month,day,hour,air_temp,dew_temp,sea_level_pressure,wind_direction,wind_speed_rate,sky_condition,precip_1hour,precip_6hour,datetime
1062,1980,2,14,6,28.0,25.0,1002.0,7.0,6.2,0.8,0.5,31.0,1980-02-14 06:00:00
1068,1980,2,14,12,28.0,26.0,999.2,2.0,3.1,0.8,0.0,31.0,1980-02-14 12:00:00
